In [11]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 400
plt.rcParams['axes.grid'] = True
plt.rcParams['image.aspect'] = 0.8

import numpy as np
import casadi as ca
import time
from helper_fns import yaml_load

def timed_exec(fn, *args):
    start = time.time()
    res = fn(*args)
    elapsed = time.time() - start
    return elapsed, *res

results = {} # value is tuple of solve time, cost

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from helper_fns import spawn_models
from mpc import MPC

mpc_params = yaml_load('config/mpc_params.yaml')
ipopt_options = yaml_load('config/ipopt_options.yaml')

_, robots, contacts = spawn_models(robot_path = "config/franka.yaml",
                                   attr_path  = "config/attrs.yaml", 
                                   contact_path = "config/contact_test.yaml",
                                   sym_vars = [])

q0 = np.ones(7)
dq0 = np.zeros(7)

params = {'q': q0,
          'dq': dq0,
          'belief_free':1.0,
          'belief_point':0.0,
          'imp_stiff':400*ca.DM.ones(3),}

In [15]:
# MPC only, i.e. without iCEM warmstart
mpc = MPC(robots, 
          mpc_params=mpc_params, 
          ipopt_options=ipopt_options,
          icem=False)

t_mpc_free_cold, cost, res = timed_exec(mpc.solve, params)
t_mpc_free_warm, cost, res = timed_exec(mpc.solve, params)

print(res.keys())
params['belief_free'] = 0.0
params['belief_point'] = 1.0

t_mpc_point, cost2, res2 = timed_exec(mpc.solve, params)

print(f"Free space cold {t_mpc_free_cold}, Free space warm {t_mpc_free_warm}, Point: {t_mpc_point}, Cost: {cost2}")
res['imp_stiff'] = params['imp_stiff']
#traj = robots['free'].get_ext_state(res)
#print(cost)
#print(traj['imp_rest'])

dict_keys(['imp_rest', 'free/q', 'free/dq', 'point/q', 'point/dq'])
Free space cold 1.0467894077301025, Free space warm 0.04230189323425293, Point: 0.012664318084716797, Cost: 0.408501


In [17]:
# iCEM-only result
mpc_params['num_iter'] = 10
mpc.icem_init()
xi0 = np.hstack((q0, dq0))
time_cem_free, cost, res = timed_exec(mpc.icem_solve, params)
time_cem_point, cost, res = timed_exec(mpc.icem_solve, params)
print(f"Free space {time_cem_free}, Point: {time_cem_point}, Cost: {cost}")


Free space 0.1268465518951416, Point: 0.1268784999847412, Cost: 1.05695


In [9]:
import matplotlib.pyplot as plt
import numpy as np

p = np.squeeze(np.array(traj['p'].T))
plt.plot(p, label=['x', 'y', 'z'])
plt.legend()

NameError: name 'traj' is not defined